# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [47]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
import json
import datetime

In [48]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [49]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [50]:
song_files = get_files("./data/song_data/")


In [51]:
filepath = song_files[0]

In [52]:
print(filepath)
df = pd.read_json(filepath,lines=True)
df.head()

/home/meep/udacity/dataEngineering/Projects/project_1/data/song_data/A/B/B/TRABBKX128F4285205.json


,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR36F9J1187FB406F1,56.27609,9.51695,Denmark,Bombay Rockers,SOBKWDJ12A8C13B2F3,Wild Rose (Back 2 Basics Mix),230.71302,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [53]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0]
song_data = list(song_data)
print(song_data)
print(type(song_data))


['SOBKWDJ12A8C13B2F3', 'Wild Rose (Back 2 Basics Mix)', 'AR36F9J1187FB406F1', 0, 230.71302]
<class 'list'>


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [54]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [55]:
artist_data = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values[0]
artist_data=list(artist_data)
print(artist_data)

['AR36F9J1187FB406F1', 'Bombay Rockers', 'Denmark', 56.27609, 9.51695]


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [56]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [57]:
log_files = get_files("./data/log_data/")

In [58]:
filepath = log_files[0]
print(filepath)

/home/meep/udacity/dataEngineering/Projects/project_1/data/log_data/2018/11/2018-11-01-events.json


In [59]:
df = pd.read_json(filepath,lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1540344794796,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [60]:
df = df.query("page == 'NextSong'")
print("Number of records:",len(df.index))
df.head()

Number of records: 11


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [61]:
t = pd.to_datetime(df["ts"],unit="ms")
print(len(t))
t.head()

11


2   2018-11-01 21:01:46.796
4   2018-11-01 21:05:52.796
5   2018-11-01 21:08:16.796
6   2018-11-01 21:11:13.796
7   2018-11-01 21:17:33.796
Name: ts, dtype: datetime64[ns]

In [62]:
time_data = map(list,zip(    
 t.tolist(),
 t.dt.hour.tolist(), 
 t.dt.day.tolist(), 
 t.dt.weekofyear.tolist(), 
 t.dt.month.tolist(), 
 t.dt.year.tolist(),
 t.dt.weekday.tolist()
 ))
time_data=list(time_data)


print(time_data)
column_labels = ["start_time", "hour", "day", "week", "month", "year", "weekday"]


[[Timestamp('2018-11-01 21:01:46.796000'), 21, 1, 44, 11, 2018, 3], [Timestamp('2018-11-01 21:05:52.796000'), 21, 1, 44, 11, 2018, 3], [Timestamp('2018-11-01 21:08:16.796000'), 21, 1, 44, 11, 2018, 3], [Timestamp('2018-11-01 21:11:13.796000'), 21, 1, 44, 11, 2018, 3], [Timestamp('2018-11-01 21:17:33.796000'), 21, 1, 44, 11, 2018, 3], [Timestamp('2018-11-01 21:24:53.796000'), 21, 1, 44, 11, 2018, 3], [Timestamp('2018-11-01 21:28:54.796000'), 21, 1, 44, 11, 2018, 3], [Timestamp('2018-11-01 21:42:00.796000'), 21, 1, 44, 11, 2018, 3], [Timestamp('2018-11-01 21:52:05.796000'), 21, 1, 44, 11, 2018, 3], [Timestamp('2018-11-01 21:55:25.796000'), 21, 1, 44, 11, 2018, 3], [Timestamp('2018-11-01 22:23:14.796000'), 22, 1, 44, 11, 2018, 3]]


/tmp/ipykernel_50688/2135722827.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  t.dt.weekofyear.tolist(),


In [63]:
print(column_labels)
time_df = pd.DataFrame(time_data, columns=column_labels)
time_df.head()


['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']


,start_time,hour,day,week,month,year,weekday
0,2018-11-01 21:01:46.796,21,1,44,11,2018,3
1,2018-11-01 21:05:52.796,21,1,44,11,2018,3
2,2018-11-01 21:08:16.796,21,1,44,11,2018,3
3,2018-11-01 21:11:13.796,21,1,44,11,2018,3
4,2018-11-01 21:17:33.796,21,1,44,11,2018,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [64]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [65]:
user_df = df[["userId","firstName","lastName","gender","level"]]
print(user_df)

    userId firstName lastName gender level
2        8    Kaylee  Summers      F  free
4        8    Kaylee  Summers      F  free
5        8    Kaylee  Summers      F  free
6        8    Kaylee  Summers      F  free
7        8    Kaylee  Summers      F  free
8        8    Kaylee  Summers      F  free
9        8    Kaylee  Summers      F  free
10      10    Sylvie     Cruz      F  free
12      26      Ryan    Smith      M  free
13      26      Ryan    Smith      M  free
14     101    Jayden      Fox      M  free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [66]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [67]:
df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [71]:
inc=1
for index, row in df.iterrows():
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    print(results)
    print(song_select)
    print(row.song,"-", row.artist,"-", row.length)
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
    
    # insert songplay record
    start_time=datetime.datetime.utcfromtimestamp(row.ts/1000.0)
    songplay_data = (inc, start_time,row.userId, row.level, songid, artistid,row.sessionId, row.location, row.userAgent)
    print(songplay_data)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()
    inc+=1

None

SELECT song_id, artists.artist_id
FROM songs
JOIN artists
ON songs.artist_id=artists.artist_id
WHERE songs.title=%s AND artists.name= %s AND songs.duration= %s

You Gotta Be - Des'ree - 246.30812
(1, datetime.datetime(2018, 11, 1, 21, 1, 46, 796000), 8, 'free', None, None, 139, 'Phoenix-Mesa-Scottsdale, AZ', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"')
None

SELECT song_id, artists.artist_id
FROM songs
JOIN artists
ON songs.artist_id=artists.artist_id
WHERE songs.title=%s AND artists.name= %s AND songs.duration= %s

Flat 55 - Mr Oizo - 144.03873
(2, datetime.datetime(2018, 11, 1, 21, 5, 52, 796000), 8, 'free', None, None, 139, 'Phoenix-Mesa-Scottsdale, AZ', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"')
None

SELECT song_id, artists.artist_id
FROM songs
JOIN artists
ON songs.artist_id=artists.artist_id
WHERE songs.title=%s AND artists.name= %s 

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [46]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.